In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gudhi as gd

In [ ]:
#LoopForest.py needs to be in same directory
from LoopForest import LoopForest

In [ ]:
%load_ext autoreload

%autoreload 3

# Example use case

In [ ]:
rng = np.random.default_rng(35)
num_points=100
points = rng.uniform(low=0.0, high=1.0, size=(num_points,2)) * 1000

plt.figure(figsize=(6,6))
plt.scatter( points[:,0], points[:,1], s = 3)
plt.show()

In [ ]:
loop_forest = LoopForest(points, reduce=True, compute_barcode=True, print_info=True)

## Barcode

The H1 barcode of the point cloud is stored in the LoopForest object loop_forest at loop_forest.barcode as a list of Bar objects. 
Each Bar object bar has attributes bar.birth, bar.death and bar.cycle_reps
The progession of optimal cycle representatives is stored in bar.cycle_reps as list of Loop objects.

Each Loop object has a vertex list which are saves as list of indices in loop.vertex_list. 
The point coordinates can be accessed via loop_forest.point_cloud[vertex] for vertex in loop.vertex_list
Additionally, each Loop has a loop.active_start and loop.active_end attribute which give the interval in which this cycle representative is optimal.

In [ ]:
for bar in list(loop_forest.barcode)[0:10]:
    print(f"\nBar atributes: Birth {bar.birth}, Death {bar.death}, \nand cycle reps")
    for loop in bar.cycle_reps:
        print(f'{loop.vertex_list} active from {loop.active_start} to {loop.active_end}')
        cycle_rep = np.array([loop_forest.point_cloud[vertex] for vertex in loop.vertex_list])

## Plotting functions for Loop Forest Class

In [ ]:
ax = loop_forest.plot_dendrogram()
plt.show()

In [ ]:
#Two Coloring schemes
#coloring = 'forest' is default and colors loops in same forest very similarly
for k in range(2,5):
    loop_forest.plot_at_filtration(50*k, figsize=(7,7))

#coloring = 'bars' cycles through different colors from longest to shortes bars
#forces long bars to have different colors even if they are in the same tree
for k in range(2,5):
    loop_forest.plot_at_filtration(50*k, figsize=(7,7),coloring='bars')

# Measurement Functions and Generalized Persistence Landscapes

In [ ]:
from point_cloud_sampling import sample_noisy_circle

circle = sample_noisy_circle(500, noise_std=0.01, seed=4, radius = 0.36)

# Visualization
plt.figure(figsize=(5,5))
plt.scatter(circle[:,0], circle[:,1], s=10, alpha=0.6)
plt.gca().set_aspect("equal")
plt.show()

In [ ]:
circle_forest = LoopForest(circle)

In [ ]:
ax = circle_forest.plot_at_filtration(0.1)
plt.show()

In [ ]:
#import polygon path measurement functions
from cycle_rep_vectorisations import polygon_area, polygon_length, polygon_length_squared_area_ratio, polygon_length_squared_area_ratio_normalized, curvature_excess

In [ ]:
#extract longest bar in barcode
bar= circle_forest.max_bar()

ax = circle_forest.plot_barcode_measurement(polyhedral_path_func=polygon_length, bar=bar) #if no bar is given, maximal bar is automatically used
plt.show()

In [ ]:
#plotting generalized landscape which is convolution of measurement function with interval indicator function
ax, _ = circle_forest.plot_generalized_interval_landscape(polyhedral_path_func=polygon_length)
plt.show()

In [ ]:
#generate plot of mutliple measurement functions and their landscapes
axes = circle_forest.plot_landscape_subplots(polyhedral_path_funcs=[polygon_area, polygon_length, polygon_length_squared_area_ratio_normalized, curvature_excess])
plt.show()

# Comparing different examples

In [ ]:
from point_cloud_sampling import sample_noisy_star

star = sample_noisy_star(1000, spikes=5, amplitude=0.5, noise_std=0.03, seed=42, radius =0.75)
star_forest = LoopForest(star)

In [ ]:
from point_cloud_sampling import sample_noisy_ellipse

ellipse = sample_noisy_ellipse(1000, a=0.9,b=0.4, noise_std = 0.03)
ellipse_forest = LoopForest(ellipse)

In [ ]:
from point_cloud_sampling import sample_noisy_circle_with_tendril

circle_line = sample_noisy_circle_with_tendril(1000, radius=0.5, tendril_length=0.3, tendril_fraction=0.05, noise_std=0.02, tendril_width_deg=2)
circle_line_forest = LoopForest(circle_line)

In [ ]:
yrange_dict = {(0,1): (0.0,2),
               (0,2):(0,11),
               (0,3):(0,60),
               (0,4): (0,40),
               (1,1): (0.0,2),
               (1,2):(0,4),
               (1,3):(0,11),
               (1,4): (0,2.5)}

column_titles = ["Area", "Length","(Length^2/Area)-4pi", "Excess Curvature"]

for forest in [circle_forest, ellipse_forest, circle_line_forest, star_forest]:
    axes = forest.plot_landscape_subplots(polyhedral_path_funcs=[polygon_area, polygon_length, polygon_length_squared_area_ratio_normalized, curvature_excess], 
                                       yrange_dict=yrange_dict, 
                                       column_titles=column_titles)
    plt.show()
